In [1]:
# 1. 라이브러리 임포트 및 버전 확인하기
import PublicDataReader as pdr
from config import API_KEY_INFO
print(pdr.__version__)

2022.07.13
['__version__', '__author__', '__contact__', '__github__', 'Transaction', 'Building', 'StoreInfo', 'Transportation', 'code_bdong']


In [2]:
# 2. 공공 데이터 포털 OpenAPI 서비스 인증키 입력하기
serviceKey = API_KEY_INFO.get("portal")

In [3]:
# 3. 국토교통부 실거래가 정보 조회 OpenAPI 세션 정의하기
# debug: True이면 모든 메시지 출력, False이면 오류 메시지만 출력 (기본값: False)
ts = pdr.Transaction(serviceKey, debug=True)

[INFO] 아파트 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[INFO] 아파트 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[ERROR] 오피스텔 매매 조회 서비스 오류 - (99) DEADLINE HAS EXPIRED ERROR.
[INFO] 오피스텔 전월세 조회 서비스 정상 - (00) NORMAL SERVICE.
[ERROR] 단독다가구 매매 조회 서비스 오류 - (99) SERVICE ACCESS DENIED ERROR.
[ERROR] 단독다가구 전월세 조회 서비스 오류 - (99) DEADLINE HAS EXPIRED ERROR.
[ERROR] 연립다세대 매매 조회 서비스 오류 - (99) DEADLINE HAS EXPIRED ERROR.
[ERROR] 연립다세대 전월세 조회 서비스 오류 - (99) DEADLINE HAS EXPIRED ERROR.
[INFO] 상업업무용 매매 조회 서비스 정상 - (00) NORMAL SERVICE.
[ERROR] 토지 매매 조회 서비스 오류 - (99) DEADLINE HAS EXPIRED ERROR.
[ERROR] 분양입주권 매매 조회 서비스 오류 - (99) DEADLINE HAS EXPIRED ERROR.
[ERROR] 공장창고등 매매 조회 서비스 오류 - (99) SERVICE ACCESS DENIED ERROR.


In [ ]:
import PublicDataReader as pdr
from config import OpenAPI
import pandas as pd
import datetime
import logging
import requests
from bs4 import BeautifulSoup

# DataFrame 디스플레이 설정
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

print(pdr.__all__)
print(pdr.__version__)
# print(pdr.__info__)

In [ ]:
# 공공 데이터 포털 서비스키
serviceKey = OpenAPI['molit']

# 실거래가
ts = pdr.Transaction(serviceKey, debug=False)
# 건축물대장
bd = pdr.Building(serviceKey, debug=False)
# 상가업소
si = pdr.StoreInfo(serviceKey, debug=False)

In [ ]:
# 서울 열린데이터 광장 서비스키
seoulServiceKey = OpenAPI['seoul']

# 지하철승하차
self = pdr.Transportation(seoulServiceKey, debug=True)

In [ ]:
category = "지하철승하차"
date = "20211101"
df = self.read_data(category, date=date)
df

In [ ]:
category = "버스승하차"
date = "20211031"
df = self.read_data(category, date=date)
df

In [ ]:
# 행정구역코드 조회
code = pdr.code_list()
code

In [ ]:
# 실거래가 조회
prod = "아파트"
trans = "매매"
sigunguCode = "41135"
startYearMonth = "202110"
endYearMonth = "202111"

df = ts.collect_data(prod, trans, sigunguCode, startYearMonth, endYearMonth)
df.tail(1)

In [ ]:
# 건축물대장 조회
category = "총괄표제부"
sigunguCd = "41135"
bjdongCd = "11000"
bun = "0542"
ji = "0000"

df = bd.read_data(category=category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)
df.tail(1)

In [ ]:
# 상가업소 조회
category = "지번상가"
key = '1165010100108120002'
indsLclsCd = 'Q'

df = si.read_data(category=category, key=key, indsLclsCd=indsLclsCd)
df.tail(1)

## 컬럼 리스트 추출

In [ ]:
# LAWD_CD=11680
# DEAL_YMD=202110


# for prod in ['아파트','오피스텔','단독다가구','연립다세대','상업업무용','분양입주권','토지']:
#     for trans in ['매매', '전월세']:
#         try:
#             print()
#             print(f">> {prod} {trans}")
#             print()
#             service_url = self.metaDict[prod][trans]['url']

#             # URL
#             url=f"""\
# {service_url}\
# &LAWD_CD={str(LAWD_CD)}\
# &DEAL_YMD={str(DEAL_YMD)}\
# &numOfRows=99999\
#             """

#             result = requests.get(url, verify=False)
#             xmlsoup = BeautifulSoup(result.text, "lxml-xml")
#             item = xmlsoup.find("item")

#             col_list = []
#             for i in item:
#                 col_list.append(i.name)
#             for col in col_list: 
#                 print(f"'{col}'", end=',')
                
#         except:
#             print("Error")
#             pass

### Features

- 공통

    - 로깅 기능
        - 세션 초기화 시 `debug` 인자 추가
            - False(Default): 오류 레벨만 출력
            - True: 모든 레벨 출력
        - OPEN API 에러 코드 출력

    - Air Korea Open API 모듈 제거
    
    - 법정동코드 테이블 조회 함수 추가

- 부동산 실거래가

    - 데이터 조회 메서드 일원화
        - molit.read_data('아파트', '매매', '11680', '202111')
            - 서비스별 End-point 및 컬럼 매핑 딕셔너리 추가
    - 데이터 수집 메서드 신규 메서드 추가
        - molit.collect_data('아파트', '매매', '11680', '202101', '202103')

- 건축물대장
    
    - 데이터 조회 메서드 일원화
        - molit.read_data(category, sigunguCd=sigunguCd, bjdongCd=bjdongCd, bun=bun, ji=ji)
        
- 서울 열린데이터 광장
    - 교통
        - 지하철 호선별 역별 승하차
        - 버스 노선별 정류장별 승하차